In [1]:
from kafka import KafkaProducer
from sseclient import SSEClient

In [2]:
producer = KafkaProducer(bootstrap_servers='sandbox.hortonworks.com:6667')
topic = 'wiki-rc-stream'
stream_src = 'https://stream.wikimedia.org/v2/stream/recentchange'

In [3]:
try:
    messages = SSEClient(stream_src)
    for msg in messages:
        producer.send(topic, msg.data.encode())
        
except Exception as e:
    sys.exit("Error producing stream: {}".format(e))

KeyboardInterrupt: 